# Bayesian inference

In [2]:
%load_ext watermark

In [224]:
import pymc as pm
import sqlite3 as sq
import pandas as pd
import itertools as itt
import arviz as az
import graphviz
import numpy as np
import importlib as imp
from collections import defaultdict

from models import OneAdvantage
from models import TeamSpecificAdvantage

import sys
sys.path.append("../")

DB_PATH = '../data/data.sqlite.db'

%watermark --iversions

sqlite3   : 2.6.0
sys       : 3.9.12 | packaged by conda-forge | (main, Mar 24 2022, 23:25:59) 
[GCC 10.3.0]
pandas    : 1.4.2
pymc      : 4.0.0b6
arviz     : 0.12.0
graphviz  : 0.19.1
matplotlib: 3.5.1
numpy     : 1.22.3



# Data preparation

First, I retrieve all games on record, generate home-away pairs and define identifiers to those pairs, and calculate correct score differences.

In [208]:
with sq.connect(DB_PATH) as conn:
    sql = """
    SELECT `home-points`, `away-points`, `home-team-id`, `away-team-id`, `start-year`, `regular` FROM games
    JOIN seasons ON games.season_id = seasons.id
    ORDER BY `game-id`
    ;
    """
    all_games = pd.read_sql_query(sql, conn)

#convert types
all_games['home-team-id']= all_games['home-team-id'].astype("int")
all_games['away-team-id']= all_games['away-team-id'].astype("int")
all_games['start-year']= all_games['start-year'].astype("int")

#apply identifiers to the dataframe
all_games['pair_id'] = all_games.apply(lambda x: (max(x['home-team-id'], x['away-team-id']), min(x['home-team-id'], x['away-team-id'])), axis=1)

#define is_home variable and set score with a correct sign
all_games['is_home'] = (all_games['home-team-id'] > all_games['away-team-id']) * 1
all_games['is_home'] = all_games['is_home'].apply(lambda x: x if x == 1 else -1)
all_games['score_diff'] = (all_games['home-points'] - all_games['away-points']) * all_games['is_home']

## Data calculation function

Function that returns computed variables used by different models.

In [211]:
def compute_data(games, cut_off_year = 2020, informative_priors = None):
    
    last_2_seasons = games[games['start-year'] >= cut_off_year]    
    pairs = pd.get_dummies(last_2_seasons['pair_id'])
    pair_vals = pairs.values
    is_home = last_2_seasons['is_home'].values.reshape(-1 , 1)
    score_diffs = last_2_seasons['score_diff'].values.reshape(-1, 1)
    is_cup = 1 - last_2_seasons['Regular'].values.reshape(-1, 1)
    no_obs, no_pairs = pair_vals.shape
    home_team_dummy = pd.get_dummies(last_2_seasons['home-team-id']).values
    away_team_dummy = pd.get_dummies(last_2_seasons['away-team-id']).values
    
    if not informative_priors:
        team_pair_advantages = np.zeros((no_pairs, 1))
        
    else:
        begin_cut_off = cut_off_year - informative_priors
        mask = (games['start-year'] < cut_off_year) & (games['start-year'] >= begin_cut_off)
        prior_seasons = games[mask]    
        average_diffs = prior_seasons[['pair_id', 'score_diff']].groupby('pair_id').mean().reset_index()

        sorted_diffs = []
        for p in pairs.columns.values:
            v = average_diffs['score_diff'][average_diffs['pair_id'] == p].values        
            if len(v) == 0:
                v = [0]
            sorted_diffs.append(v)

        team_pair_advantages = np.fromiter(itt.chain(*sorted_diffs), dtype=float).reshape(-1, 1)
        
    
    return {
        'pair_vals' : pair_vals,
        'pair_ids': pairs.columns.values,
        'is_game_home': is_home,
        'score_diffs': score_diffs,
        'is_cup': is_cup,
        'no_obs': no_obs,
        'no_pairs': no_pairs,
        'home_teams': home_team_dummy,
        'away_teams': away_team_dummy,
        'no_home_teams': home_team_dummy.shape[1],
        'no_away_teams': away_team_dummy.shape[1],
        'pair_priors' : team_pair_advantages,        
    }


## Model definitions

Two versions are created: 
 - using uninformative priors for team strenghts
 - using the average pair-wise score from 2 seasons prior the data used as the prior for team strenghts

### Model #1 - Homecourt advantage is a overall phenomenon

In [225]:
models = defaultdict(dict)

In [226]:
imp.reload(OneAdvantage)
with pm.Model() as model:
    data = compute_data(all_games, cut_off_year=2020, informative_priors=None)    
    OneAdvantage.OneAdvantage(data = data)
    models['single_flat_model']['pair_ids'] = data['pair_ids']
    models['single_flat_model']['model'] = model

with pm.Model() as model:
    data = compute_data(all_games, cut_off_year=2020, informative_priors=2)    
    OneAdvantage.OneAdvantage(data = data)
    models['single_empirical_model']['pair_ids'] = data['pair_ids']
    models['single_empirical_model']['model'] = model

### Model #2 - homecourt advantage varies by team

In [227]:
imp.reload(TeamSpecificAdvantage)
with pm.Model() as model:
    data = compute_data(all_games, cut_off_year=2020, informative_priors=None)    
    TeamSpecificAdvantage.TeamSpecificAdvantage(data = data)
    models['team_flat_model']['pair_ids'] = data['pair_ids']
    models['team_flat_model']['model'] = model

with pm.Model() as model:
    data = compute_data(all_games, cut_off_year=2020, informative_priors=2)    
    TeamSpecificAdvantage.TeamSpecificAdvantage(data = data)
    models['team_empirical_model']['pair_ids'] = data['pair_ids']
    models['team_empirical_model']['model'] = model

## Visualize and save the models

In [229]:
pm.model_graph.model_to_graphviz(models['team_flat_model']['model']).render("../results/team-model", format='png')
pm.model_graph.model_to_graphviz(models['single_flat_model']['model']).render("../results/single-model", format='png')

#to adjust ratio: dot -Tpng -Gratio=0.7 -Gdpi=100 -ofoo.png team-flat

'../results/single-model.png'

## MCMC

Run MCMC and save the traces

In [232]:
for name, model in models.items():
    print(name)
    trace = pm.sample(
        draws=2_000,        
        tune=500,
        cores=6,        
        chains=4,        
    )
    az.to_netcdf(trace, "../traces/{}.netcdf".format(name))

single_flat_model
single_empirical_model
team_flat_model
team_empirical_model
